In [1]:
import pandas as pd
import pyodbc
import numpy as np
import os

In [22]:
path = r'T:\Tableau\tableauJourneyToWork\Datasources'

In [37]:
# adjust accordingly on the tables ModeShare_ALL_Years; ModeByVehiclesAvailable_AllYears
data = pd.read_csv(path + '\\ModeByPovertyStatus_AllYears.csv')   
df = pd.DataFrame(data)

print(df)

      Year              Geography  \
0     2009                 Coburg   
1     2009                 Coburg   
2     2009                 Coburg   
3     2009                 Coburg   
4     2009                 Coburg   
...    ...                    ...   
1165  2021  Eugene Urbanized Area   
1166  2021  Eugene Urbanized Area   
1167  2021   Salem Urbanized Area   
1168  2021   Salem Urbanized Area   
1169  2021   Salem Urbanized Area   

                                       Travel.Mode  \
0                 Car, truck, or van - drove alone   
1                   Car, truck, or van - carpooled   
2        Public transportation (excluding taxicab)   
3                                           Walked   
4     Taxicab, motorcycle, bicycle, or other means   
...                                            ...   
1165                                Worked at home   
1166                                Worked at home   
1167                                Worked at home   
1168           

In [44]:
df.dtypes

Year                int64
Geography          object
Travel.Mode        object
Poverty.Status     object
Estimate            int64
MOE_Est             int64
Share             float64
MOE_Share         float64
SharePCT          float64
MOE_SharePct      float64
dtype: object

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [38]:
df['Travel.Mode'].unique()

array(['Car, truck, or van - drove alone',
       'Car, truck, or van - carpooled',
       'Public transportation (excluding taxicab)', 'Walked',
       'Taxicab, motorcycle, bicycle, or other means', 'Worked at home'],
      dtype=object)

In [39]:
len('Taxicab, motorcycle, bicycle, or other means')

44

In [40]:
df['Poverty.Status'].unique()

array(['100 to 149 percent of the poverty level',
       'At or above 150 percent of the poverty level',
       'Below 100 percent of the poverty level'], dtype=object)

In [41]:
len('At or above 150 percent of the poverty level')

44

In [42]:
df.replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)

In [43]:
df = df.fillna(0)

In [45]:
# table names: mode_share, mode_by_vehicles_available
# rewrite tables: DROP TABLE mode_share;
# edit accordingly the table
cursor.execute('''
            CREATE TABLE mode_by_poverty_status(
            year int,
            geo varchar(50),
            mode varchar(50),
            poverty varchar(50),
            estimate int,
            moe_est int,
            share float,
            moe_share float,
            share_pct float,
            moe_share_pct float
            )
               ''')

In [46]:
#df.rename(columns={'Mode.by.Vehicle.Availability': 'Mode', 'Vehicles.Available': 'VHCL_AVAIL'}, inplace=True)
df.rename(columns={'Travel.Mode': 'Mode', 'Poverty.Status': 'Poverty_Status'}, inplace=True)

In [47]:
# mode_share, mode_by_vehicles_available
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO mode_by_poverty_status(year, geo, mode, poverty, estimate, moe_est, share, moe_share, share_pct, moe_share_pct)
                VALUES (?,?,?,?,?,?,?,?,?,?)
                ''',
                   row.Year,
                   row.Geography,
                   row.Mode,
                   row.Poverty_Status,
                   row.Estimate,
                   row.MOE_Est,
                   row.Share,
                   row.MOE_Share,
                   row.SharePCT,
                   row.MOE_SharePct
                  )
conn.commit()